## MRI File Transfer (.sh)

You should save this in the root directory, or one above the directory you are mounting. Use the number key to select the mounted directory you are trying to pull the files from. 

*Remember to replace with actual path of your scratch folder*

This version finds recent directories so it must be run on the same day. Name this script something like MRI_RECENT. Use the 2nd version if doing mutiple or specific transfers. 

VERSION: `MRI_Auto_Sync`

In [ ]:
#!/bin/bash

echo "Found directories:"

parent_dirs=(*/)
for i in "${!parent_dirs[@]}"; do
    echo "[$((i+1))] ${parent_dirs[$i]}"
done

read -p "Choose a directory (by number): " parent_choice
parent_dir_index=$((parent_choice-1))

if [[ $parent_dir_index -lt 0 || $parent_dir_index -ge ${#parent_dirs[@]} ]]; then
    echo "Invalid selection. Exiting."
    exit 1
fi

selected_parent_dir="${parent_dirs[$parent_dir_index]}"
echo "Selected directory: $selected_parent_dir"

echo "Looking for recent directories in $selected_parent_dir"
recent_dirs=($(find "$selected_parent_dir" -maxdepth 1 -mindepth 1 -type d -mmin -$((10*60)) -printf '%T+ %p\n' | sort -r | cut -d" " -f2-))

if [ ${#recent_dirs[@]} -eq 0 ]; then
    echo "No recent directories found in $selected_parent_dir."
    exit 1
fi

echo "Found recent directories:"
for i in "${!recent_dirs[@]}"; do
    echo "[$((i+1))] ${recent_dirs[$i]}"
done

read -p "Choose one of the recent directories (by number): " recent_choice
recent_dir_index=$((recent_choice-1))

if [[ $recent_dir_index -lt 0 || $recent_dir_index -ge ${#recent_dirs[@]} ]]; then
    echo "Invalid selection. Exiting."
    exit 1
fi

selected_recent_dir="${recent_dirs[$recent_dir_index]}"
echo "Selected recent directory: $selected_recent_dir"

echo "Pulling information..."
num_files=$(find "$selected_recent_dir" -type f | wc -l)
size=$(du -sh "$selected_recent_dir" | cut -f1)
echo "Size of folder: $size"
echo "Number of files: $num_files"

read -p "Do you want to send this folder? (Y/N) " confirmation
if [[ $confirmation == [Yy] ]]; then
    scp -r "$selected_recent_dir" "samhsu03@graham.computecanada.ca:/home/samhsu03/scratch/MRINEW/"
    echo "Folder sent."
else
    echo "Transfer cancelled."
fi


This version will promt you to enter the name of the directory you want to transfer. Name this script something like MRI_SYNC.

VERSION: `MRI_Manual_Sync`

In [ ]:
#!/bin/bash

echo "Found directories:"

dirs=(*/)
for i in "${!dirs[@]}"; do
    echo "[$((i+1))] ${dirs[$i]}"
done

read -p "Choose a directory (by number): " choice
dir_index=$((choice-1))

if [[ $dir_index -lt 0 || $dir_index -ge ${#dirs[@]} ]]; then
    echo "Invalid selection. Exiting."
    exit 1
fi

selected_dir="${dirs[$dir_index]}"

read -p "Name of session folder: " session_folder

session_path="${selected_dir}${session_folder}"

if [ ! -d "$session_path" ]; then
    echo "Error: Directory '$session_path' does not exist."
    exit 1
fi

echo "Pulling information..."
num_files=$(find "$session_path" -type f | wc -l)
size=$(du -sh "$session_path" | cut -f1)
echo "Size of folder: $size"
echo "Number of files: $num_files"

read -p "Do you want to send this folder? (Y/N) " confirmation
if [[ $confirmation == [Yy] ]]; then
    scp -r "$session_path" "samhsu03@graham.computecanada.ca:/home/samhsu03/scratch/MRINEW/"
    echo "Session folder $session_folder sent."
else
    echo "Transfer cancelled."
fi
